<a href="https://colab.research.google.com/github/GurkiratSarna/CE888-Decision-Making-UoE/blob/master/Lab_8/TransferLearning_Example1_cats_and_dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Transfer Learning using Cats and Dogs dataset

In [0]:
import os
#!rm -r CE888_2020
#!rm -rf data

In [0]:
import os
os.environ['KAGGLE_USERNAME'] = "gurkiratsarna" # username from the json file
os.environ['KAGGLE_KEY'] = "704aca2803d9d723440ca20977f69914" # key from the json file
!kaggle competitions download -c dogs-vs-cats # api copied from kaggle

  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 73.6MB/s]
 99% 540M/543M [00:03<00:00, 163MB/s]
100% 543M/543M [00:03<00:00, 179MB/s]
 98% 265M/271M [00:01<00:00, 148MB/s]
100% 271M/271M [00:02<00:00, 140MB/s]


In [0]:
# Unzip the data from train.zip to current directory /content
from zipfile import ZipFile

file_name = "/content/train.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('done')

done


In [0]:
!pip install tensorflow==1.3.0
!pip install keras==2.0.7

     |████████████████████████████████| 43.6MB 93kB/s 
     |████████████████████████████████| 1.6MB 52.3MB/s 
     |████████████████████████████████| 890kB 42.5MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107220 sha256=97ab41dbf33663bcf4496d4bda6d723ce46d34baed42bb0fb8ce6dbf14c5010f
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
ERROR: stable-baselines 2.2.1 has requirement tensorflow>=1.5.0, but you'll have tensorflow 1.3.0 which is incompatible.
ERROR: magenta 0.3.19 has requirement tensorflow>=1.12.0, but you'll have tensorflow 1.3.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 1.3.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
  Found existing installation: html5l

In [0]:
import numpy as np
import os
import time
from CE888_2020.Lab_8.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.applications.imagenet_utils import decode_predictions
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [16]:
#%% Load an elephant image as an example and see the image-processing

img_path = '/content/train/cat.1.jpg' # image name

img = image.load_img(img_path, target_size=(224, 224)) # load image of size (img_height, img_width)

x = image.img_to_array(img) # convert image to array
print (x.shape)

x = np.expand_dims(x, axis=0) # expand dimension
print (x.shape)

x = preprocess_input(x)
print('Input image shape:', x.shape) # the first number is the number of images = 1 here.

(224, 224, 3)
(1, 224, 224, 3)
Input image shape: (1, 224, 224, 3)


In [17]:
#%% Loading the training data
PATH = os.getcwd()
PATH

'/content'

Create two folders for cats and dogs.

In [0]:
#Create a data folder that will contain 2 folders - cats and dogs
original_dataset_dir = '/content/train'
base_dir = '/content/data'
os.mkdir(base_dir)

In [0]:
#create cats and dogs folder under data folder
dogs_dir = os.path.join(base_dir, 'dogs')
os.mkdir(dogs_dir)

cats_dir = os.path.join(base_dir, 'cats')
os.mkdir(cats_dir)

In [0]:
# Define data path
data_path = PATH + '/data'
data_dir_list = os.listdir(data_path)
data_dir_list

['horses', 'dogs', 'Humans', 'cats']

In [0]:
img_data_list=[] # created an empty list for images to be loaded